## **INTEGRANTE:**
Giovanna Fernandes Pereira | RM: 565434

*   PARTE 01 - exs: 11, 12, 13, 14 e 15
*   PARTE 02 - exs: 21, 22, 23, 24 e 25

# **Importação de pacotes**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

# **Criando um dataframe com base non arquivo .csv externo**

In [ ]:
df = pd.read_csv('/content/household_power_consumption.txt', sep=';')

# **Análise Exploratória**

In [ ]:
# Visualizar os primeiros registros (linhas) -----> atributos (colunas)
df.head()

In [ ]:
# Nome dos atributos (colunas)
df.columns

In [ ]:
# Tamanho do dataframe
df.shape

In [ ]:
print(f'O dataframe tem {df.shape[0]} linhas e {df.shape[1]} colunas')

In [ ]:
# Retorna as características das colunas
df.info()

In [ ]:
# object ------> colunas com dados categóricos (tipicamente String)
# int e float ----> colunas com dados numéricos/contínuos

In [ ]:
# Descrição estatística das colunas numéricas
df.describe()

# **PARTE 01 - Data Science e Machine Learning no Python e Orange Data Mining**

## **11. Calcule a correlação entre as variáveis Global_active_power, Global_reactive_power, Voltage e Global_intensity.**

In [ ]:
variaveis = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']
for col in variaveis:
   df[col] = pd.to_numeric(df[col], errors='coerce')
display(df[variaveis].corr())

# OU

# variaveis = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']
# df[variaveis].corr()

## **12. Crie uma nova variável chamada Total_Sub_metering que some Sub_metering_1, Sub_metering_2 e Sub_metering_3.**

In [ ]:
df['Sub_metering_1'] = pd.to_numeric(df['Sub_metering_1'], errors='coerce')
df['Sub_metering_2'] = pd.to_numeric(df['Sub_metering_2'], errors='coerce')
df['Total_Sub_metering'] = df['Sub_metering_1'] + df['Sub_metering_2'] + df['Sub_metering_3']
display(df['Total_Sub_metering'].head())

# OU

# df['Total_Sub_metering'] = df['Sub_metering_1'] + df['Sub_metering_2'] + df['Sub_metering_3']
# df['Total_Sub_metering'].head()

## **13. Verifique se há algum mês em que Total_Sub_metering ultrapassa a média de Global_active_power.**

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
df['AnoMes'] = df['Date'].dt.to_period('M')
media_global = df['Global_active_power'].mean()
meses_total = df.groupby('AnoMes')['Total_Sub_metering'].mean()
meses_acima_media = meses_total[meses_total > media_global]
display(meses_acima_media)

# OU

# media_global = df['Global_active_power'].mean()
# meses_total = df.groupby('AnoMes')['Total_Sub_metering'].mean()
# meses_acima_media = meses_total[meses_total > media_global]
# display(meses_acima_media)

## **14. Faça um gráfico de série temporal do Voltage para o ano de 2008.**

In [ ]:
df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])

df_2008 = df[df['DateTime'].dt.year == 2008]

plt.figure(figsize=(15, 6))
plt.plot(df_2008['DateTime'], df_2008['Voltage'])
plt.title('Time Series of Voltage for 2008')
plt.xlabel('Date and Time')
plt.ylabel('Voltage')
plt.grid(True)
plt.show()

# OU

# df_2008 = df[df['Date'].dt.year == 2008]
# df_2008.set_index('Date')['Voltage'].plot()
# plt.title('Série Temporal do Voltage - Ano 2008')
# plt.ylabel('Voltage')
# plt.show()

## **15. Compare o consumo entre os meses de verão e inverno (no hemisfério norte).**

In [ ]:
df['Mes'] = df['Date'].dt.month
verao   = df[df['Mes'].isin([6, 7, 8])]
inverno = df[df['Mes'].isin([12, 1, 2])]
verao_mean   = verao['Global_active_power'].mean()
inverno_mean = inverno['Global_active_power'].mean()
print(f'Média de Global_active_power no verão: {verao_mean:}')          # OU -> {verao_mean:.2f}
print(f'Média de Global_active_power no inverno: {inverno_mean:}')      # OU -> {inverno_mean:.2f}

# **PARTE 02 – Exercícios adicionais no dataset inicial**

## **21. Séries temporais por hora**

In [ ]:
df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
df = df.set_index('Datetime')
consumo_horario = df['Global_active_power'].resample('H').mean()
consumo_por_hora = consumo_horario.groupby(consumo_horario.index.hour).mean()
display(consumo_por_hora.sort_index().to_frame(name='Média de Consumo por Hora'))

display(consumo_por_hora.sort_values(ascending=False).head())

# OU

# df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
# df = df.set_index('Datetime')
# consumo_horario = df['Global_active_power'].resample('H').mean()
# consumo_por_hora = consumo_horario.groupby(consumo_horario.index.hour).mean()
# print(consumo_por_hora.sort_values(ascending=False).head())

## **22. Autocorrelação do consumo**

In [ ]:
serie = consumo_horario.dropna()

autocorr_1h = serie.autocorr(lag=1)
autocorr_24h = serie.autocorr(lag=24)
autocorr_48h = serie.autocorr(lag=48)

print("Autocorrelação 1h:", autocorr_1h)
print("Autocorrelação 24h:", autocorr_24h)
print("Autocorrelação 48h:", autocorr_48h)

### **Pergunta: existem padrões repetidos diariamente?**

Com base nos resultados da autocorrelação:

*   A **autocorrelação em 1 hora (0.71)** é relativamente alta, o que indica uma forte correlação positiva entre o consumo em uma hora e o consumo na hora anterior. Isso é esperado para dados de séries temporais onde os valores tendem a ser semelhantes aos valores passados recentes.
*   A **autocorrelação em 24 horas (0.44)** também é positiva e moderadamente alta. Isso sugere que realmente existem padrões diários repetidos nos dados. O consumo em uma determinada hora é correlacionado com o consumo na mesma hora do dia anterior.
*   A **autocorrelação em 48 horas (0.40)** é ligeiramente menor do que em 24 horas, mas ainda positiva. Isso sugere que o padrão ainda está um pouco presente após dois dias, mas menos pronunciado do que o padrão diário.

Portanto, os resultados da autocorrelação, particularmente o lag de 24 horas, indicam a existência de padrões diários repetidos nos dados de consumo de energia.

## **23. Redução de dimensionalidade com PCA**

In [ ]:
X = df[['Global_active_power','Global_reactive_power','Voltage','Global_intensity']].dropna()
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
print("Variância explicada por componente:", pca.explained_variance_ratio_)

## **24. Visualização de clusters no espaço PCA**

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_pca)

plt.figure(figsize=(8,5))
plt.scatter(X_pca[:,0], X_pca[:,1], c=clusters, cmap='viridis', alpha=0.5)
plt.title("Clusters no espaço PCA (2 componentes)")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.show()

### **Pergunta: os grupos se separam de forma clara?**

Com base na visualização dos clusters no espaço PCA:

*   Os clusters não se separam de forma completamente clara, mas há uma distinção visível.
*   Um dos clusters (o grupo verde no gráfico) parece estar mais claramente separado dos outros dois.
*   Os outros dois clusters (os grupos amarelo e roxo no gráfico) mostram uma sobreposição considerável.

Portanto, enquanto o PCA ajudou a identificar alguns padrões de agrupamento nos dados, os clusters não são totalmente distintos no espaço bidimensional reduzido.

## **25. Regressão polinomial vs linear**

In [ ]:
X = df[['Voltage']].dropna()
y = df.loc[X.index, 'Global_active_power']

lin_model = LinearRegression()
lin_model.fit(X, y)
y_pred_lin = lin_model.predict(X)

rmse_lin = np.sqrt(mean_squared_error(y, y_pred_lin))


poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
poly_model = LinearRegression()
poly_model.fit(X_poly, y)
y_pred_poly = poly_model.predict(X_poly)


rmse_poly = np.sqrt(mean_squared_error(y, y_pred_poly))

print(f"RMSE Linear: {rmse_lin}")
print(f"RMSE Polinomial (grau 2): {rmse_poly}")


plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.3, label="Dados reais", s=5)


X_plot = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
X_plot_poly = poly.transform(X_plot)
y_plot_poly = poly_model.predict(X_plot_poly)

plt.plot(X, y_pred_lin, color='red', label="Regressão Linear")
plt.plot(X_plot, y_plot_poly, color='green', label="Regressão Polinomial (grau 2)")

plt.title("Regressão Polinomial vs Linear (Global_active_power vs Voltage)")
plt.xlabel("Voltage")
plt.ylabel("Global_active_power")
plt.legend()
plt.grid(True)
plt.show()